### Install `mmk` and its dependencies

In [ ]:
!git clone https://github.com/antoinedaurat/mmk.git
!pip install -r mmk/requirements.txt

### load the api token of your neptune account

In [ ]:
from getpass import getpass
api_token = getpass('Enter your private Neptune API token: ')

In [ ]:
%cd ../..

### Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
plt.rcParams['figure.figsize'] = (20, 6)
import torch
import torch.nn as nn
import numpy as np
import os

# class defining the class of FreqNet models and the dictionary to subclass it
# from mmk.models.freqnet import FreqNet, layer_funcs
# from mmk.models.model_base import Model

# generate function 
# from mmk.modules.generate import generate

# data utils
# from mmk.data import Database

# debug, interact
# from mmk.utils import show, audio, download_database, download_model, upload_model

import pytorch_lightning as pl

In [ ]:
################## NEW model_base.py 
# to maximize compatibility and modularity
# we only subclass a few things :

import os
from mmk.kit import MMKHooks, EpochEndPrintHook, get_trainer, \
    MMKCheckpoint, MMKDefaultLogger, EpochProgressBarCallback

## TODO : Callback that log audio and spectro to neptune (on_val_epoch_end? on_save_checkpoint)        

        
## Example Model / Sketch for a unit-test : 
    
class TestModel(MMKHooks, EpochEndPrintHook, pl.LightningModule):
    
    def __init__(self,**some_params):
        super(TestModel, self).__init__()
        self.save_hyperparameters()
        self.fc = None  # set in the next method!
        
    def setup(self, stage):
        if stage == "fit":
        # here, the trainer is attached to self, hence also the datamodule
        # this is where lightning recommands to do data-dependant configurations like :
            input_dim = 14
            self.fc = nn.Linear(input_dim, input_dim)
        
    def forward(self, x):
        return self.fc(x)
    
    def training_step(self, batch, batch_idx):
        inpt, target = batch
        pred = self.forward(inpt)
        L = nn.MSELoss()(pred, target)
        self.log("recon", L, on_step=False, on_epoch=True)
        return {"loss": L, "recon_loss": L.clone()}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def train_dataloader(self):
        ds = torch.utils.data.TensorDataset(torch.randn(200, 14), torch.randn(200, 14))
        return torch.utils.data.DataLoader(ds, batch_size=5)
    
    def val_dataloader(self):
        ds = torch.utils.data.TensorDataset(torch.randn(200, 14), torch.randn(200, 14))
        return torch.utils.data.DataLoader(ds, batch_size=5)

        
tm = TestModel(hi=1, unstringable=EpochEndPrintHook)

root_dir = "pl_test/"

trainer = pl.Trainer(default_root_dir=root_dir,
                     gpus=int(torch.cuda.is_available()),
                     checkpoint_callback=MMKCheckpoint(dirpath=root_dir, epochs=2),
                     logger=MMKDefaultLogger(root_dir, version=None),
                     max_epochs=20,
                     callbacks=[EpochProgressBarCallback()],
                     check_val_every_n_epoch=1,
                    )

trainer.fit(tm)

### Resume Training

In [ ]:
trainer = pl.Trainer(resume_from_checkpoint="pl_test/states/epoch=19.ckpt",
                     default_root_dir=root_dir,
                     gpus=int(torch.cuda.is_available()),
                     checkpoint_callback=MMKCheckpoint(dirpath=root_dir, epochs=2),
                     logger=MMKDefaultLogger(root_dir, version=None),
                     max_epochs=20,
                     callbacks=[EpochProgressBarCallback()],
                     check_val_every_n_epoch=1,
                    )

trainer.fit(tm)

### `MMKDefaultLogger` subclasses the `pl.loggers.TestTubeLogger` which is based on Tensorboard :

In [ ]:
!tensorboard --logdir "pl_test/logs/"

### peek into a checkpoint : 

In [ ]:
ckpt_path = "pl_test/states/epoch=19.ckpt"
torch.load(ckpt_path)

### Sketch to get the audio logged in a Tensorboard as numpy array (currently no audio is logged but this is fairly easy...)

In [ ]:
from tensorboard.backend import event_processing as ep 
import tensorflow as tf

print(os.listdir("pl_test/logs/tf/"))

acc = ep.event_accumulator.EventAccumulator("pl_test/logs/tf/events.out.tfevents.1606509829.MrGr.7090.2")
acc.Reload()

# Print tags of contained entities, use these names to retrieve entities as below
print(acc.Tags())

tf.audio.decode_wav(acc.Audio("audio_13.wav")[0].encoded_audio_string).audio.numpy()


## Neptune stuff that I used for logging audio, tagging experiment in k-tonal/Bruckner

In [ ]:
from neptune import Session

session = Session.with_default_backend(api_token=api_token)
exps = session.get_project("k-tonal/Bruckner").get_experiments()
to_tag = [e for e in exps if int(e.id.split('-')[-1]) > 27]

for exp in to_tag:
    params = exp.get_parameters()
    for tag in exp.get_tags():
        exp.remove_tag(tag)
    
    exp.append_tag("HK")
    
    if params["hk_overlap"] == 'True':
        exp.append_tag("conjunct_kernel")
    else:
        exp.append_tag("disjunct_kernel")
    
    if "accum_outputs=1" in params["lf"]:
        exp.append_tag("residual_left")
    else:
        exp.append_tag("residual_right")
        
    if params["model_dim"] == 256:
        exp.append_tag("small")
    elif params["model_dim"] == 1024:
        exp.append_tag("medium")
    
    if "2" in params["layers"]:
        exp.append_tag("short")
    else:
        exp.append_tag("long")

In [ ]:
import soundfile as sf
from neptunecontrib.api.audio import log_audio
from neptunecontrib.api.chart import log_chart
import shutil


# download_model(api_token, "k-tonal/Bruckner", exps[0].id)
# net = FreqNet.load(FreqNet, "root/")
# net.prepare_data()
# inpt, _ = next(iter(net.train_dataloader()))
# shutil.rmtree("root/")

for exp in exps:
    
    if int(exp.id.split("-")[-1]) < 34:
        continue
        
    print("Updating experiment", exp.id)
    
    download_model(api_token, "k-tonal/Bruckner", exp.id)
    
    if "HK" in exp.get_tags():
        net = HKFreqNet.load(HKFreqNet, "root/")
    else:
        net = FreqNet.load(FreqNet, "root/")

    for i in range(4):
        Y = generate(net.to("cuda"), inpt.to("cuda")[i], 2048, *net.generation_slices()).T
        y = signal(Y)
        sf.write("root/generated_%i.wav" % (i+1), y, 22050, subtype="PCM_24")
        log_audio("root/generated_%i.wav" % (i+1), "generated_%i" % (i+1), exp)
        fig = plt.figure()
        show(Y)
        log_chart("generated_%i" % (i+1), fig, exp)
    del net
    shutil.rmtree("root/")